In [108]:
# Common DS imports
import pandas as pd
import numpy as np

#Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

#Helpers
import wrangle #'./wrangle.py'

# DS Pipeline step 1: Acquire
This section will serve as documentation for the process of getting the data into a python environment.  At the end of this section each step will be compiled and documented into a set of functions whose purpose will be to make repeating the process as simple as possible.

### Requirements
The process will make use of libraries not included with the base installation of python.  

In [109]:
# """
# These commands will install the listed libraries in your python environment.
# Since they are only helpful if libraries are missing, to avoid accidentally running them they have been commented out.
# If you want to run these commands they will first need to be uncommented (Delete the '#' character at the start of each line)
# """

# %pip install numpy
# %pip install pandas

### Setting up the SQL code
The data for this project lives on CodeUp's MySQL server, so I will need to use python to connect to the server and download the data.  Luckily, pandas includes a method specefically meant for this scenario: `pd.read_sql()`

The hardest part of using this method is formatting the URL used in the method without exposing my credentials.  To get around that, I'll store my credentials in a secret `env.py` file and write a function that can be used to create the URL dynamically.  I happen to have one ready-made so there's no need for me to re-invent the wheel.

In [110]:
def get_db_url(database, hostname='', username='', password='', env=''):
    '''Creates a URL for a specific database and credential set to be used with pymysql.

    Can be used either with a set of credentials passed directly to the function or with an environment file containing the credentials.
    If both are provided, the environment file takes precedence.

    Parameters:
    database (str): The target database that pymysql will connect to, which will provide context for any SQL queries used in the connection.
    hostname (str): The DNS hostname or IP-Adress for the connection
    username (str), password (str): User credentials that will be used in a sql connection.
    env (str): Relative path to an environment file. ('./env.py')  The file must include the hostname, username, and password variables.

    Returns:
    str: Full URL for use with a pymysql connection
    '''
    if env != '':
        d = {}
        file = open(env)
        for line in file:
            (key, value) = line.split('=')
            d[key] = value.replace('\n', '').replace("'",'').replace('"','')
        username = d['username']
        hostname = d['hostname']
        password = d['password']
    url = f'mysql+pymysql://{username}:{password}@{hostname}/{database}'
    return url


Now that the function has been declared we can feed it into the `pandas.read_sql()` method and start pulling some data.

In [111]:
# Use our get_db_url() function to create a properly formatted url and store it into its own variable to make things easier to read later.
url = get_db_url('telco_churn', env='./env.py')

# SQL syntax can get unruly, and this particular query isn't exactly straightforward. So we'll store it in its own variable as well.
query = """
        SELECT 
            *
        FROM
            telco_churn.customers
                JOIN
            telco_churn.internet_service_types USING (internet_service_type_id)
                JOIN
            telco_churn.payment_types USING (payment_type_id)
                JOIN
            telco_churn.contract_types USING (contract_type_id)
        ;
        """

# Now it's time to use pandas to import our data.  
df = pd.read_sql(query, url)

# Display the first two rows to make sure the import was successful
df.head(2)

,contract_type_id,payment_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,payment_type,contract_type
0,2,2,1,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,DSL,Mailed check,One year
1,1,2,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,DSL,Mailed check,Month-to-month


Next we'll package this process up into its own function and commit it to `wrangle.py`

In [112]:
def new_telco_data():
    url = get_db_url('telco_churn',env='./env.py')
    query = """
        SELECT 
            *
        FROM
            telco_churn.customers
                JOIN
            telco_churn.internet_service_types USING (internet_service_type_id)
                JOIN
            telco_churn.payment_types USING (payment_type_id)
                JOIN
            telco_churn.contract_types USING (contract_type_id)
        ;
        """
    df = pd.read_sql(query, url)
    return df

# Let's make sure it works
new_telco_data().head(2)

,contract_type_id,payment_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,payment_type,contract_type
0,2,2,1,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,DSL,Mailed check,One year
1,1,2,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,DSL,Mailed check,Month-to-month


In [113]:
# And also make sure it works when imported from our helper library
wrangle.new_telco_data().head(2)

,contract_type_id,payment_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,payment_type,contract_type
0,2,2,1,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,DSL,Mailed check,One year
1,1,2,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,DSL,Mailed check,Month-to-month


### Cacheing the data
Now that we have the data in hand I should store it locally.  This will speed things up by not having to download a new copy of the data each time.  It also ensures that we can still access the data in the event of a server outage.

Pandas helps us again with its `pandas.to_csv()` method.

In [114]:
# Store the data in a csv file in this directory with the name telco.csv


# df.to_csv('./telco.csv') #Commented out to reduce write-cycles on my system's drive

Of course, this only helps us if we can read the data from the CSV file as well.  We'll do so with `pandas.read_csv()`

In [115]:
# Read the data from our newly created csv file.
pd.read_csv('./telco.csv').head(2)

,Unnamed: 0,contract_type_id,payment_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,payment_type,contract_type
0,0,2,2,1,0002-ORFBO,Female,0,Yes,Yes,9,...,Yes,Yes,No,Yes,65.6,593.3,No,DSL,Mailed check,One year
1,1,1,2,1,0003-MKNFE,Male,0,No,No,9,...,No,No,Yes,No,59.9,542.4,No,DSL,Mailed check,Month-to-month


Hmm.. It looks like there's a new column there that we don't want: `Unnamed: 0`.  This happens because when we asked pandas to write our data to a file, it also included the dataframe index.
Luckily we can tell pandas that our file includes an index by passing in the argument `index_col=`. 

In [116]:
# Read the data again, this time declaring which column to use as the dataframe index.
pd.read_csv('./telco.csv', index_col=0).head(2)

,contract_type_id,payment_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,payment_type,contract_type
0,2,2,1,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,DSL,Mailed check,One year
1,1,2,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,DSL,Mailed check,Month-to-month


Great!  But we also don't want to have to keep track of whether or not there's already a csv file cached and which method to use to acquire the data.  We'll create a function for that:

In [117]:
# This function makes use of the `os` library.  We'll have to import it first.
import os

def get_telco_data():
    filename = "telco.csv"
    
    # if file is available locally, read it
    if os.path.isfile(filename):
        return pd.read_csv(filename, index_col=0)
    
    # if file not available locally, acquire data from SQL database
    # and write it as csv locally for future use
    else:
        # read the SQL query into a dataframe
        df = new_telco_data()
        
        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename)

        # Return the dataframe to the calling code
        return df  

# Make sure it works
get_telco_data().head(2)

,contract_type_id,payment_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,payment_type,contract_type
0,2,2,1,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,DSL,Mailed check,One year
1,1,2,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,DSL,Mailed check,Month-to-month


In [118]:
# And also make sure it works from our helper library
df = wrangle.get_telco_data()
df.head(2)

,contract_type_id,payment_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,payment_type,contract_type
0,2,2,1,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,DSL,Mailed check,One year
1,1,2,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,DSL,Mailed check,Month-to-month


# DS Pipeline Step 2: Preparation

There's a lot of columns here, and the dataframe is too wide for our notebook to display it properly. We will want to think about trimming this down, so we'll use python to generate a list programatically and also get some basic information about each column.

The first method we'll use is `.info()` which gives us a lot of useful information in one go
 - The number of entries (rows)
 - The number of features (columns)
 - The number of non-null values in each column
 - The data type (Dtype) of each column

In [119]:
# Use DataFrame.info() to get a list of columns.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   contract_type_id          7043 non-null   int64  
 1   payment_type_id           7043 non-null   int64  
 2   internet_service_type_id  7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

The first thing that stands out to me is there seem to be some columns (`'contract_type_id', 'payment_type_id', 'internet_service_type_id'`) that are essentially duplicated.  In fact, we know that they are because they were the columns we used to join when we wrote the SQL query.  We can safely drop those columns.

In [120]:
# Drop columns that just repeat the data in other columns
df = df.drop(columns=['contract_type_id', 'payment_type_id', 'internet_service_type_id'])
print( df.columns.to_list() )

['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents', 'tenure', 'phone_service', 'multiple_lines', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'paperless_billing', 'monthly_charges', 'total_charges', 'churn', 'internet_service_type', 'payment_type', 'contract_type']


Also from running `.info()` we can see that we are working with 7043 entries. It looks like none of them have null values, but we can't always trust that for its face value.  Stepping through each column and seeing what data lives in them will give us more insight into whethere there are any 'functionally null' values, like blank strings or strings meant to represent a null value in the data.  

In this case, because we don't have a data dictionary provided for us and will need to infer it ourselves, going through each column will also give us some insight into what each column is meant to represent.

In [121]:
# To make the cell output more manageable later I will focus on the columns that are likely to have a lot of different values first.
df.customer_id.value_counts()

0002-ORFBO    1
6616-AALSR    1
6625-UTXEW    1
6625-IUTTT    1
6625-FLENO    1
             ..
3352-RICWQ    1
3352-ALMCK    1
3351-NQLDI    1
3351-NGXYI    1
9995-HOTOH    1
Name: customer_id, Length: 7043, dtype: int64

There doesn't appear to be any null values here, and we can see that each row seems to be unique so we can go ahead and use the column's name to infer the first column of our data dictionary.
> **customer_id**: A unique identifier that represents a single customer

In [122]:
df.tenure.value_counts()

1     613
72    362
2     238
3     200
4     176
     ... 
28     57
39     56
44     51
36     50
0      11
Name: tenure, Length: 73, dtype: int64

This one requires a bit more thought to decipher.  Looking up the dictionary definition for "tenure" we can find a defintion of "a period for which an office is held".  Because none of the other definitions make sense for a telecom company and wouldn't be stored as an integer it can be assumed that tenure is meant to be a measure of how long a customer has been with the company.

Since I don't imagine a single customer having a tenure of 613 years, and it would be resource intensive to update a database each day with the new values, we can infer this columns as:
> **tenure**: The number of months a customer has been a customer.

In [123]:
df.monthly_charges.value_counts()

20.05     61
19.85     45
19.95     44
19.90     44
20.00     43
          ..
116.55     1
106.85     1
68.55      1
108.80     1
67.85      1
Name: monthly_charges, Length: 1585, dtype: int64

This is a float variable with lots of different values.  It's safe to assume it is what it says on the tin:
> **monthly_charges**: The amount a customer is charged each month

In [124]:
df.total_charges.value_counts()

           11
20.2       11
19.75       9
19.9        8
20.05       8
           ..
2387.75     1
6302.8      1
2058.5      1
829.55      1
3707.6      1
Name: total_charges, Length: 6531, dtype: int64

**Hold up!**  
It's easy to miss it, but did you see what happened here?  
There are 11 values that appear to be blank.  This is because SQL returned those null values as empty strings, which is technically not a null value.

Because these 11 rows are such a small part of our dataset it's easy to say that we can just drop them.  But I'd like to take a closer look at them to see if there is any information to be gleaned from this abnormality, especially since the rest of the dataset seems so well maintained.

In [125]:
# Get a list of all customers where total charges is a single space
mask = (df.total_charges == ' ')
df[mask]

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,payment_type,contract_type
945,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,Yes,Yes,...,Yes,Yes,No,No,56.05,,No,DSL,Credit card (automatic),Two year
1731,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No internet service,No internet service,...,No internet service,No internet service,No internet service,No,20.00,,No,None,Mailed check,Two year
1906,2775-SEFEE,Male,0,No,Yes,0,Yes,Yes,Yes,Yes,...,Yes,No,No,Yes,61.90,,No,DSL,Bank transfer (automatic),Two year
2025,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No internet service,No internet service,...,No internet service,No internet service,No internet service,Yes,19.70,,No,None,Mailed check,One year
2176,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No internet service,No internet service,...,No internet service,No internet service,No internet service,No,20.25,,No,None,Mailed check,Two year
2250,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No internet service,No internet service,...,No internet service,No internet service,No internet service,No,25.35,,No,None,Mailed check,Two year
2855,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,No,Yes,...,Yes,Yes,No,No,73.35,,No,DSL,Mailed check,Two year
3052,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No internet service,No internet service,...,No internet service,No internet service,No internet service,No,25.75,,No,None,Mailed check,Two year
3118,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,Yes,No,...,Yes,Yes,No,Yes,52.55,,No,DSL,Bank transfer (automatic),Two year
4054,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,Yes,Yes,...,No,Yes,Yes,No,80.85,,No,DSL,Mailed check,Two year


Each of these customers have a tenure of 0.  It makes sense that their total charges would be 0 if they haven't been around long enough to be charged.  
Do these customers make up the entirety of the new customers?

In [126]:
# Get the number of customers with 0 tenure.
(df.tenure == 0).sum()

11

Yes they do.  To fix this we can replace the missing values with 0.

In [127]:
# Replaces the spaces in total_charges with a 0 and sets the column's type to a float
df.total_charges = df.total_charges.replace(' ', 0).astype(np.float64)

And finally we can infer the description for this column:
> **total_charges**: The total amount that has been charged to a customer over their tenure

There are still a lot of columns remaining, and doing them all one at a time would take a long time.  To save time we'll loop through all of the remaining columns and look at their value counts.

In [128]:
columns_to_skip = ['customer_id','tenure','monthly_charges','total_charges']

#Loop through each column in the dataframe, and if we haven't done them already, print their value counts.
for col in df:
    if col not in columns_to_skip:
        print(df[col].value_counts(), '\n')

Male      3555
Female    3488
Name: gender, dtype: int64 

0    5901
1    1142
Name: senior_citizen, dtype: int64 

No     3641
Yes    3402
Name: partner, dtype: int64 

No     4933
Yes    2110
Name: dependents, dtype: int64 

Yes    6361
No      682
Name: phone_service, dtype: int64 

No                  3390
Yes                 2971
No phone service     682
Name: multiple_lines, dtype: int64 

No                     3498
Yes                    2019
No internet service    1526
Name: online_security, dtype: int64 

No                     3088
Yes                    2429
No internet service    1526
Name: online_backup, dtype: int64 

No                     3095
Yes                    2422
No internet service    1526
Name: device_protection, dtype: int64 

No                     3473
Yes                    2044
No internet service    1526
Name: tech_support, dtype: int64 

No                     2810
Yes                    2707
No internet service    1526
Name: streaming_tv, dtype: int64

Most of the remaining columns can be retyped into booleans, which tend to work better with machine learning models.  There are some columns that in addition to the Yes/No values there is a value determining whether the customer has the appropriate internet service or phone service for the column to be applicable.  However, we can already infer that information from the `phone_service` and `internet_service_type` columns.  Therefore we can convert all of the remaining columns, with the exception of `internet_service_type`, `payment_type`, and `contract_type` into boolean columns.

In [129]:
# Convert the senior citizen column into boolean type.
# We want to handle this one separetely because unlike the others, it uses 1's and 0's instead of Yes/No
df.senior_citizen = df.senior_citizen == 1
# Do something similar with gender
df.gender = df.gender == 'Male'
# Also rename the gender column to make it more intuitive to read later.
df.rename(columns={'gender': 'is_male'}, inplace=True)


In [130]:
# Loop through the yes/no columns and convert them to booleans.
yes_no_columns = [ 'partner',
                    'dependents',
                    'phone_service',
                    'multiple_lines',
                    'online_security',
                    'online_backup',
                    'device_protection',
                    'tech_support',
                    'streaming_tv',
                    'streaming_movies',
                    'paperless_billing',
                    'churn'
                    ]
for column in yes_no_columns:
    df[column] = df[column] == "Yes"

In [133]:
# Make sure all of the transformations look how we expect
df.head(5)

,customer_id,is_male,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,payment_type,contract_type
0,0002-ORFBO,False,False,True,True,9,True,False,False,True,...,True,True,False,True,65.6,593.30,False,DSL,Mailed check,One year
1,0003-MKNFE,True,False,False,False,9,True,True,False,False,...,False,False,True,False,59.9,542.40,False,DSL,Mailed check,Month-to-month
2,0004-TLHLJ,True,False,False,False,4,True,False,False,False,...,False,False,False,True,73.9,280.85,True,Fiber optic,Electronic check,Month-to-month
3,0011-IGKFF,True,True,True,False,13,True,False,False,True,...,False,True,True,True,98.0,1237.85,True,Fiber optic,Electronic check,Month-to-month
4,0013-EXCHZ,False,True,True,False,3,True,False,False,False,...,True,True,False,True,83.9,267.40,True,Fiber optic,Mailed check,Month-to-month


So far we've made a fair amount of transformations to the data and now we have a dataframe that's ready for exploration.  Let's take a moment to compile into a function and add it to the `wrangle.py` file.

In [134]:
def get_tidy_telco_data():
    # Get a fresh copy of the data
    df = get_telco_data()
    # Drop columns that just repeat the data in other columns
    df = df.drop(columns=['contract_type_id', 'payment_type_id', 'internet_service_type_id'])
    # Replace the spaces in total_charges with a 0 and sets the column's type to a float
    df.total_charges = df.total_charges.replace(' ', 0).astype(np.float64)
    # Convert the senior citizen column into boolean type.
    # We want to handle this one separetely because unlike the others, it uses 1's and 0's instead of Yes/No
    df.senior_citizen = df.senior_citizen == 1
    # Do something similar with gender
    df.gender = df.gender == 'Male'
    # Also rename the gender column to make it more intuitive to read later.
    df.rename(columns={'gender': 'is_male'}, inplace=True)
    # Loop through the yes/no columns and convert them to booleans.
    yes_no_columns = [ 'partner',
                    'dependents',
                    'phone_service',
                    'multiple_lines',
                    'online_security',
                    'online_backup',
                    'device_protection',
                    'tech_support',
                    'streaming_tv',
                    'streaming_movies',
                    'paperless_billing',
                    'churn']
    for column in yes_no_columns:
        df[column] = df[column] == "Yes"
    return df

# Test functionality
get_tidy_telco_data().head(2)    

,customer_id,is_male,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,payment_type,contract_type
0,0002-ORFBO,False,False,True,True,9,True,False,False,True,...,True,True,False,True,65.6,593.3,False,DSL,Mailed check,One year
1,0003-MKNFE,True,False,False,False,9,True,True,False,False,...,False,False,True,False,59.9,542.4,False,DSL,Mailed check,Month-to-month


Because we know we are going to be making machine learning models from the data, it is a good idea to create dummy columns (a set of boolean columns where each column corresponds to each of the possible categories in a different column) for the categorical variables.  Pandas includes a method called `get_dummies` that simplifies the process.



However, carrying those columns around during exploration can be unweildy so we'll pack that process into a separate function that we can call later when it's time to mess with modeling.

In [136]:
# Demonstration of the get_dummies() method
pd.get_dummies(df, columns=['internet_service_type', 'payment_type', 'contract_type'], dtype=bool).head(2)

,customer_id,is_male,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,internet_service_type_DSL,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Bank transfer (automatic),payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check,contract_type_Month-to-month,contract_type_One year,contract_type_Two year
0,0002-ORFBO,False,False,True,True,9,True,False,False,True,...,True,False,False,False,False,False,True,False,True,False
1,0003-MKNFE,True,False,False,False,9,True,True,False,False,...,True,False,False,False,False,False,True,True,False,False


In [137]:
def get_tidier_telco_data():
    # Start with all of the tidying we did earlier
    df = get_tidy_telco_data()
    # And tack some dummy columns onto it
    return pd.get_dummies(df, columns=['internet_service_type', 'payment_type', 'contract_type'], dtype=bool)

# Test the new function
get_tidier_telco_data().head(2)

,customer_id,is_male,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,internet_service_type_DSL,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Bank transfer (automatic),payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check,contract_type_Month-to-month,contract_type_One year,contract_type_Two year
0,0002-ORFBO,False,False,True,True,9,True,False,False,True,...,True,False,False,False,False,False,True,False,True,False
1,0003-MKNFE,True,False,False,False,9,True,True,False,False,...,True,False,False,False,False,False,True,True,False,False
